In [5]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize

# 1. 데이터 불러오기
raw_path = '../../0_data/raw/TS2000_RAW.xlsx'
df = pd.read_excel(raw_path)

# 2. 행 기준 결측치 80% 이상이면 해당 행 제거
df = df[df.isnull().mean(axis=1) < 0.8]

# 3. 결측치 중앙값으로 대체 (수치형 컬럼만)
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isna().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# 4. 윈저라이징 (상하위 0.5%)
for col in numeric_cols:
    mask = df[col].notna()
    if mask.sum() > 0:
        df.loc[mask, col] = winsorize(df.loc[mask, col], limits=[0.005, 0.005])

# 5. 상장폐지일 → is_defaulted (1/0), 이후 원본 컬럼 제거
if '상장폐지일' in df.columns:
    df['is_defaulted'] = df['상장폐지일'].notnull().astype(int)
    df.drop(columns='상장폐지일', inplace=True)

# 6. 저장
output_path = 'preprocessed_data.csv'
df.to_csv(output_path, index=False)
print(f"✅ 전처리 완료: {output_path}")



✅ 전처리 완료: preprocessed_data.csv
